## Recommender Systems

In [297]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, SVD, Dataset,evaluate

In [298]:
import warnings; warnings.simplefilter('ignore')

# Simple Recommender

In [299]:
md = pd.read_csv('movies_metadata.csv')

In [300]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [301]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

In [302]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')

In [303]:
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')

In [304]:
C = vote_averages.mean()

In [305]:
C

5.244896612406511

In [306]:
m = vote_counts.quantile(0.95)

In [307]:
m

434.0

In [308]:
md['year'] = pd.to_datetime(md['release_date'],errors='coerce').apply(lambda x: str(x).split('-')[0] if x!=np.nan else np.nan)

In [309]:
md[['release_date','year']].head()

,release_date,year
0,1995-10-30,1995
1,1995-12-15,1995
2,1995-12-22,1995
3,1995-12-22,1995
4,1995-02-10,1995


In [310]:
qualified = md[(md['vote_count']>=m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genres']]

In [311]:
qualified.head()

,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415.0,7.7,21.9469,"[Animation, Comedy, Family]"
1,Jumanji,1995,2413.0,6.9,17.0155,"[Adventure, Fantasy, Family]"
5,Heat,1995,1886.0,7.7,17.9249,"[Action, Crime, Drama, Thriller]"
9,GoldenEye,1995,1194.0,6.6,14.686,"[Adventure, Action, Thriller]"
15,Casino,1995,1343.0,7.8,10.1374,"[Drama, Crime]"


In [312]:
qualified['vote_count'] = qualified['vote_count'].astype('int') 

In [313]:
qualified['vote_average'] = qualified['vote_average'].astype('int') 

In [314]:
qualified.shape

(2274, 6)

In [315]:
def weighted_rating(x):
    v = x['vote_count']
    w = x['vote_average']
    return (v/(v+m)*w)+(m/(m+v)*C)

In [316]:
qualified['wr'] = qualified.apply(weighted_rating,axis=1)

In [317]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

# Top Movies

In [318]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


In [319]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1,drop=True)

In [320]:
s.name = 'genre'

In [321]:
gen_md = md.drop('genres',axis=1).join(s)

In [423]:
gen_md.head()

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy


In [323]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

# Top Romance Movies

In [324]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


# Content Based Filtering


In [325]:
links_small = pd.read_csv('links_small.csv')

In [326]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [327]:
links_small.shape

(9125, 3)

In [328]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [329]:
md = md.drop([19730,29503,35587])

In [330]:
md['id'] = md['id'].astype('int')

In [331]:
smd = md[md['id'].isin(links_small)]

In [332]:
smd.shape

(9099, 25)

# Movie Description Based Recommender

In [333]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [334]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')

In [335]:
tfidf_matrix = tf.fit_transform(smd['description'])

In [336]:
tfidf_matrix.shape

(9099, 268124)

In [337]:
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

In [338]:
cosine_sim.shape

(9099, 9099)

In [339]:
smd  = smd.reset_index()

In [340]:
titles = smd['title']

In [341]:
indices = pd.Series(smd.index,index = smd['title'])

In [342]:
def get_recommendations(title):
    idx  = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores,key = lambda x: x[1],reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [343]:
get_recommendations('The Godfather').head()

973     The Godfather: Part II
8387                The Family
3509                      Made
4196        Johnny Dangerously
29              Shanghai Triad
Name: title, dtype: object

In [344]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

# Metadata Based Recommender

In [345]:
credits = pd.read_csv('credits.csv')

In [346]:
keywords = pd.read_csv('keywords.csv')

In [347]:
credits.shape

(45476, 3)

In [348]:
keywords.shape

(46419, 2)

In [349]:
keywords['id'] = keywords['id'].astype('int')

In [350]:
credits['id'] = credits['id'].astype('int')

In [351]:
md['id'] = md['id'].astype('int')

In [352]:
md.shape

(45463, 25)

In [353]:
md = md.merge(credits,on='id')

In [354]:
md.shape

(45538, 27)

In [355]:
md = md.merge(keywords,on='id')

In [356]:
md.shape

(46628, 28)

In [357]:
smd = md[md['id'].isin(links_small)]

In [358]:
smd.shape

(9219, 28)

In [359]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [360]:
smd['cast'] = smd['cast'].apply(literal_eval)

In [361]:
smd['keywords'] = smd['keywords'].apply(literal_eval)

In [362]:
smd['crew'] = smd['crew'].apply(literal_eval)

In [364]:
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))

In [365]:
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [366]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [367]:
smd['director'] = smd['crew'].apply(get_director)

In [368]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance (x,list) else [])

In [369]:
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >= 3 else x)

In [370]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance (x,list) else [])

In [371]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

In [372]:
smd['director'] = smd['director'].astype('str').apply(lambda x: [str.lower(x.replace(" ",""))])

In [373]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1,drop=True)

In [374]:
s.name = 'keyword'

In [375]:
s = s.value_counts()

In [376]:
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [377]:
s = s[s>1]

In [378]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [379]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [380]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)

In [381]:
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

In [382]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director']*3 + smd['genres']


In [383]:
smd['soup'][0]

['jealousi',
 'toy',
 'boy',
 'friendship',
 'friend',
 'rivalri',
 'boynextdoor',
 'newtoy',
 'toycomestolif',
 'tomhanks',
 'timallen',
 'donrickles',
 'johnlasseter',
 'johnlasseter',
 'johnlasseter',
 'Animation',
 'Comedy',
 'Family']

In [384]:
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [386]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [387]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [388]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [389]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [390]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

In [391]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [392]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
132,Batman Forever,1529,5,1995,5.054144
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


In [393]:
improved_recommendations('Mean Girls')

,title,vote_count,vote_average,year,wr
1547,The Breakfast Club,2189,7,1985,6.709602
390,Dazed and Confused,588,7,1993,6.254682
8883,The DUFF,1372,6,2015,5.818541
3712,The Princess Diaries,1063,6,2001,5.781086
4763,Freaky Friday,919,6,2003,5.757786
6277,Just Like Heaven,595,6,2005,5.681521
6959,The Spiderwick Chronicles,593,6,2008,5.680901
7494,American Pie Presents: The Book of Love,454,5,2009,5.119690
7332,Ghosts of Girlfriends Past,716,5,2009,5.092422
7905,Mr. Popper's Penguins,775,5,2011,5.087912


# Collaborative Filtering

In [394]:
reader = Reader()

In [395]:
ratings = pd.read_csv('ratings_small.csv')

In [396]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [397]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

In [398]:
data.split(n_folds=5)

In [399]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8915
MAE:  0.6869
------------
Fold 2
RMSE: 0.8948
MAE:  0.6878
------------
Fold 3
RMSE: 0.9018
MAE:  0.6952
------------
Fold 4
RMSE: 0.8970
MAE:  0.6893
------------
Fold 5
RMSE: 0.8967
MAE:  0.6922
------------
------------
Mean RMSE: 0.8963
Mean MAE : 0.6903
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.891454202874623,
                             0.8947858932986512,
                             0.9017715282111588,
                             0.897015428761388,
                             0.8966633342892707],
                            'mae': [0.6869490533414881,
                             0.6878067572630238,
                             0.6952417760621739,
                             0.6893476769128678,
                             0.6921855599106534]})

In [400]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [401]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [402]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.5379206755245916, details={'was_impossible': False})

# Hybrid Recommender

In [403]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [405]:
id_map = pd.read_csv('links_small.csv')[['movieId','tmdbId']]

In [406]:
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)

In [407]:
id_map.columns = ['movieId','id']

In [415]:
id_map = id_map.merge(smd[['title','id']],on='id').set_index('title')

In [416]:
indices_map = id_map.set_index('id')

In [420]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [421]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
1011,The Terminator,4208.0,7.4,1984,218,3.354831
974,Aliens,3282.0,7.7,1986,679,3.196708
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,3.122603
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.071847
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.974087
2014,Fantastic Planet,140.0,7.6,1973,16306,2.883798
922,The Abyss,822.0,7.1,1989,2756,2.845302
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.745632
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.670047
1376,Titanic,7770.0,7.5,1997,597,2.655697


In [422]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.372100
344,True Lies,1138.0,6.8,1994,36955,3.284496
2014,Fantastic Planet,140.0,7.6,1973,16306,3.272576
974,Aliens,3282.0,7.7,1986,679,3.195584
1011,The Terminator,4208.0,7.4,1984,218,3.190875
2132,Superman II,642.0,6.5,1980,8536,3.093297
7265,Dragonball Evolution,475.0,2.9,2009,14164,3.079297
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.026148
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.005829
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.962772
